In [1]:
import pandas as pd
import numpy as np
import math
import arviz as az
import csv
import os

In [2]:
def read_in_forward_migration_rates(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "state" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if ("region.rates" in col) or ("region.meanRate" in col):
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                            name = line.split("\t")[n]
                            deme = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                            #interval = name.split(".")[2]

                            mig_key[n] = name
                            mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])
                    
                
                
                
    return(mig_rates_dict)

In [3]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    count = 0
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "rates" in i:
            origin = i.split(".")[2]
            destination = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]




            try:
                local_df = pd.DataFrame.from_dict({"origin": origin, "destination":destination, "mean_migration":mean_percent, 
                                                   "upper_hpd_95":upper_hpd_log_95,"lower_hpd_95":[lower_hpd_log_95], 
                                                   "upper_hpd_50":upper_hpd_log_50,"lower_hpd_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            #count +=1  
    return(new_df)

In [4]:
def extract_clock_rate(input_df):
    for i in input_df.columns.tolist():
        if "Rate" in i:
            local_series = input_df[i].astype('float').to_numpy()
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]

    return(mean_percent)

In [8]:
#depending on the analysis, need to add subfolder name. i.e. "preprint_symmetrical"
log_file_path = "../results/"


In [9]:
for name in os.listdir(log_file_path):
    if "beast" in name:
        #print(name)
        file_path = os.path.join(log_file_path, name)
        # Open file
        with open(file_path) as file:
            file_name_parts = name.split(".")[0:2]
            new_name = "migration_rates_" + file_name_parts[0] + file_name_parts[1]
            print(new_name)

migration_rates_beast_rep-2-seq-density-01-biased-estimated-symmetrical
migration_rates_beast_rep-2-seq-density-01-unbiased-estimated-symmetrical
migration_rates_beast_rep-1-seq-density-01-unbiased-estimated-symmetrical
migration_rates_beast_rep-1-seq-density-01-biased-estimated-symmetrical


In [11]:
mig_dict = {}
burnin_percent = 0.15
for name in os.listdir(log_file_path):
    if "beast" in name:
        print(name)
        file_path = os.path.join(log_file_path, name)
        # Open file
        with open(file_path) as file:
            file_name_parts = name.split(".")[0:2]
            new_name = "migration_rates_" + file_name_parts[0] + file_name_parts[1]

            foo = read_in_forward_migration_rates(file_path)
            temp_df = pd.DataFrame.from_dict(foo)
            rows_to_remove = int(len(temp_df)* burnin_percent)
            mig_dict[new_name] = temp_df.iloc[rows_to_remove:]


beast_rep-2-seq-density-0.1-biased-estimated-symmetrical.log
beast_rep-2-seq-density-0.1-unbiased-estimated-symmetrical.log
beast_rep-1-seq-density-0.1-unbiased-estimated-symmetrical.log
beast_rep-1-seq-density-0.1-biased-estimated-symmetrical.log


In [12]:
mig_dict

{'migration_rates_beast_rep-2-seq-density-01-biased-estimated-symmetrical':           sample        region.meanRate      region.rates.0.1  \
 6617    66170000   0.004397130191711979   0.07692685113626722   
 6618    66180000  0.0037746109529811995   0.26140258322332866   
 6619    66190000  0.0046103506940933375    0.2557374738476457   
 6620    66200000   0.004593208691788131    1.1238841161789987   
 6621    66210000  0.0045088368933654775   0.05696743784703082   
 ...          ...                    ...                   ...   
 44110  441100000  0.0032170496090434325  0.004825463049410579   
 44111  441110000   0.004258080134025822   0.17750641466315517   
 44112  441120000  0.0030337015592469714   0.03174303369562628   
 44113  441130000  0.0031525921626758017   0.06185375884908655   
 44114  441140000   0.003567355320430866    0.1440738620244228   
 
           region.rates.0.2     region.rates.0.3      region.rates.0.4  \
 6617    0.3896961256279922   0.6323933920646342    0.077

In [13]:
mig_rates_summary_dict = {}
for key, value in mig_dict.items():
    mig_rates_summary_dict[key] = generate_summary_df(value)


/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_33363/2824630834.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_33363/2824630834.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_33363/2824630834.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_df)
/var/folders/h9/0347drb57tsgxzbddmvmlvf80000gp/T/ipykernel_33363/2824630834.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(local_

In [14]:
clock_rate_dict = {}
for key, value in mig_dict.items():
    clock_rate_dict[key] = extract_clock_rate(value)


In [15]:
clock_rate_dict

{'migration_rates_beast_rep-2-seq-density-01-biased-estimated-symmetrical': 0.003657631128974944,
 'migration_rates_beast_rep-2-seq-density-01-unbiased-estimated-symmetrical': 0.00435801543742263,
 'migration_rates_beast_rep-1-seq-density-01-unbiased-estimated-symmetrical': 0.0017306965490557751,
 'migration_rates_beast_rep-1-seq-density-01-biased-estimated-symmetrical': 0.0019718849469185715}

In [16]:
for key, value in mig_rates_summary_dict.items():
    mig_rates_summary_dict[key]["instant_rates"] = mig_rates_summary_dict[key].mean_migration * clock_rate_dict[key]
    mig_rates_summary_dict[key]["instant_rates_upper_95hpd"] = mig_rates_summary_dict[key].upper_hpd_95 * clock_rate_dict[key]
    mig_rates_summary_dict[key]["instant_rates_lower_95hpd"] = mig_rates_summary_dict[key].lower_hpd_95 * clock_rate_dict[key]


In [18]:
mig_rates_summary_dict["migration_rates_beast_rep-1-seq-density-01-biased-estimated-symmetrical"].instant_rates.to_numpy()

array([0.00176628, 0.00166442, 0.0016348 , 0.00188123, 0.00125109,
       0.0017606 , 0.00145108, 0.00307627, 0.00316558, 0.00197042])

In [19]:
mig_rates_summary_dict["migration_rates_beast_rep-1-seq-density-01-biased-estimated-symmetrical"]

,origin,destination,mean_migration,upper_hpd_95,lower_hpd_95,upper_hpd_50,lower_hpd_50,instant_rates,instant_rates_upper_95hpd,instant_rates_lower_95hpd
0,0,1,0.895732,2.813025,0.000064,0.581344,0.000104,0.001766,0.005547,1.263444e-07
0,0,2,0.844078,2.738457,0.000008,0.524583,0.000008,0.001664,0.005400,1.568855e-08
0,0,3,0.829054,1.813814,0.111705,0.851864,0.330202,0.001635,0.003577,2.202685e-04
0,0,4,0.954024,2.932021,0.000024,0.644843,0.000188,0.001881,0.005782,4.745434e-08
0,1,2,0.634463,1.531418,0.006089,0.599641,0.162631,0.001251,0.003020,1.200681e-05
0,1,3,0.892849,2.874451,0.000011,0.559817,0.000011,0.001761,0.005668,2.157517e-08
0,1,4,0.735887,2.105387,0.000022,0.585724,0.068236,0.001451,0.004152,4.376777e-08
0,2,3,1.560066,3.152087,0.323664,1.618668,0.700080,0.003076,0.006216,6.382282e-04
0,2,4,1.605360,3.182695,0.336515,1.728465,0.815859,0.003166,0.006276,6.635695e-04
0,3,4,0.999258,2.178848,0.111742,1.007906,0.368791,0.001970,0.004296,2.203431e-04


In [15]:
mig_rates_summary_dict["migration_rates_5_demes_biased"]

KeyError: 'migration_rates_5_demes_biased'

In [20]:
list(mig_rates_summary_dict["migration_rates_beast_rep-1-seq-density-01-unbiased-estimated-symmetrical"].mean_migration - mig_rates_summary_dict["migration_rates_beast_rep-1-seq-density-01-biased-estimated-symmetrical"].mean_migration)

[0.3222497709189679,
 0.019722003848881897,
 0.0729991915717989,
 -0.06634213818720536,
 0.10234278739724834,
 0.09513747845777021,
 0.49367402960324624,
 -0.6127497698758926,
 -0.776855241378704,
 0.32080998073321376]

In [21]:
for key, value in mig_rates_summary_dict.items():
    value.to_csv(key+".csv")